<a href="https://colab.research.google.com/github/ucfilho/AI/blob/main/AI_2023/CV_CNN/class_005_tune_hyperbandKeras_CNN_blood_cells_may_15_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import tensorflow as tf
tf.__version__

'2.12.0'

In [32]:
# !pip install optuna

In [33]:
from tensorboard.plugins.hparams import api as hp

In [34]:
#import optuna

In [35]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Conv2D, BatchNormalization
from tensorflow.keras.layers import  Dropout,ZeroPadding2D, MaxPooling2D, Activation, add
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

import cv2
import numpy as np

from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt

from glob import glob

import zipfile
import pandas as pd


from sklearn.metrics import classification_report, confusion_matrix

In [36]:
IMAGE_SIZE = [224,224]
ImgSize = IMAGE_SIZE[0]
epochs = 16
batch_size = 128

In [37]:
!gdown 1CU4UkqJae07u1tKfp2rajWwwZA0SKiIm

Downloading...
From: https://drive.google.com/uc?id=1CU4UkqJae07u1tKfp2rajWwwZA0SKiIm
To: /content/blood_cell_images.zip
100% 110M/110M [00:00<00:00, 188MB/s]


In [38]:
FILE='blood_cell_images.zip'

In [39]:
archive = zipfile.ZipFile(FILE, 'r')
archive.extractall()

In [40]:
PARAMS = {'lr':0.005, 
          'epochs':10,
          'batch_size':32}


In [41]:
!ls

blood_cell_images      __MACOSX     untitled_project
blood_cell_images.zip  sample_data


In [42]:
train_path = 'blood_cell_images/TRAIN'
test_path = 'blood_cell_images/TEST'
images_test = glob(test_path + '/*/*.jp*g')
images_train = glob(train_path + '/*/*.jp*g')

In [43]:
folders_train = glob(train_path + '/*')
folders_test = glob(test_path + '/*')

In [44]:
folders_train

['blood_cell_images/TRAIN/LYMPHOCYTE',
 'blood_cell_images/TRAIN/MONOCYTE',
 'blood_cell_images/TRAIN/EOSINOPHIL',
 'blood_cell_images/TRAIN/NEUTROPHIL']

In [45]:
folders_test

['blood_cell_images/TEST/LYMPHOCYTE',
 'blood_cell_images/TEST/MONOCYTE',
 'blood_cell_images/TEST/EOSINOPHIL',
 'blood_cell_images/TEST/NEUTROPHIL']

In [46]:
ntrain, ntest = len(images_train),len(images_test)
ntrain,ntest

(9957, 2487)

In [47]:
images_train[:4]
num_classes = 4

In [48]:
types = ['NEUTROPHIL', 'EOSINOPHIL', 'MONOCYTE', 'LYMPHOCYTE']

In [49]:
size = 20 # resize the images considering this value
n_epochs = 1000

In [50]:

def load_images(images,types=types, dim = (size,size)):

  nsize = len(images)
  X =[]
  y =[]
  names =[]
  k = 0

  for i in range(nsize): 
    img = image.load_img(images[i])
    im = cv2.resize(np.array(img), dim)
    im = preprocess_input(im)
    k += 1
    if( k % 3 == 0):
      X.append(im)

      if(types[0] in images[i]):
        names.append(types[0])
        y.append(0)
      elif(types[1] in images[i]):
        names.append(types[1])
        y.append(1)
      elif(types[2] in images[i]):
        names.append(types[2])
        y.append(2)
      else:
        names.append(types[3])
        y.append(3)

  X =np.array(X)
  return X,y,names

In [51]:

X_train, y_train, names_train = load_images(images_train)
X_test, y_test, names_test = load_images(images_test)

In [52]:
X_train[0].shape

(20, 20, 3)

In [53]:
df = pd.DataFrame({'name':names_train,'class':y_train})
y_train = pd.get_dummies(df['class'])
y_train

,0,1,2,3
0,0,0,0,1
1,0,0,0,1
2,0,0,0,1
3,0,0,0,1
4,0,0,0,1
...,...,...,...,...
3314,1,0,0,0
3315,1,0,0,0
3316,1,0,0,0
3317,1,0,0,0


In [54]:
for i in range(4): 
  df_plot= df[(df["class"] == i)]
  n= df_plot.shape[0]
  print(types[i],n)

NEUTROPHIL 833
EOSINOPHIL 833
MONOCYTE 826
LYMPHOCYTE 827


In [55]:
df = pd.DataFrame({'name':names_test,'class':y_test})
y_test = pd.get_dummies(df['class'])
y_test

,0,1,2,3
0,0,0,0,1
1,0,0,0,1
2,0,0,0,1
3,0,0,0,1
4,0,0,0,1
...,...,...,...,...
824,1,0,0,0
825,1,0,0,0
826,1,0,0,0
827,1,0,0,0


In [56]:
'''
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.2, 0.5))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd','RMSprop']))
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([ 1e-3, 1e-4]))
HP_EPOCHS = hp.HParam('epochs', hp.Discrete([100, 200]))
METRIC_ACCURACY = 'accuracy''''

In [69]:
def build_model(hp):
  DROPOUT = hp.Float('dropout', 0, 0.5, step=0.1, default=0.5)
  HP_OPTIMIZER = hp.Choice('OPTIMIZER',['adam', 'sgd','RMSprop'])
  HP_LEARNING_RATE = hp.Choice('LR',[1e-3, 1e-4])
  HP_EPOCHS = hp.Choice('EPOCHS',[100, 200])
  model = Sequential()
  model.add(Conv2D(32, (3, 3), padding='same',
                  input_shape=X_train.shape[1:]))
  model.add(Activation('relu'))
  model.add(Conv2D(32, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(DROPOUT))

  model.add(Conv2D(64, (3, 3), padding='same'))
  model.add(Activation('relu'))
  model.add(Conv2D(64, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(DROPOUT))

  model.add(Flatten())
  model.add(Dense(512))
  model.add(Activation('relu'))
  model.add(Dropout(DROPOUT))
  model.add(Dense(num_classes))
  model.add(Activation('softmax'))

  if 'adam'== HP_OPTIMIZER:
    optimizer = optimizer=tf.keras.optimizers.Adam(learning_rate=HP_LEARNING_RATE )
  elif 'sgd'== HP_OPTIMIZER:
    optimizer = optimizer=tf.keras.optimizers.SGD(learning_rate=HP_LEARNING_RATE )
  else: # 'RMSprop'
    optimizer = optimizer=tf.keras.optimizers.RMSprop(learning_rate=HP_LEARNING_RATE /10)

  #optimizer=hparams[HP_OPTIMIZER]
  #optimizer=hparams[HP_OPTIMIZER](learning_rate = hparams[HP_LEARNING_RATE])
  #optimizer= tf.keras.optimizers.RMSprop(learning_rate=hp_learning_rate)

  model.compile(optimizer=optimizer,
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
                metrics=['accuracy'])  
  
 
  return model

In [58]:
#!pip install kerastuner
!pip install keras-tuner --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [59]:
# https://blog.tensorflow.org/2020/01/hyperparameter-tuning-with-keras-tuner.html

In [70]:
import keras_tuner as kt

tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=200,
    hyperband_iterations=2)

In [ ]:
tuner.search(X_train,y_train,
             validation_data=(X_test,y_test),
             epochs=30,
             callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)])

Trial 49 Complete [00h 00m 06s]
val_accuracy: 0.258142352104187

Best val_accuracy So Far: 0.6598311066627502
Total elapsed time: 00h 05m 09s

Search: Running Trial #50

Value             |Best Value So Far |Hyperparameter
0.3               |0.2               |dropout
RMSprop           |adam              |OPTIMIZER
0.001             |0.001             |LR
200               |200               |EPOCHS
3                 |3                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
4                 |4                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/3
104/104 [==============================] - 3s 10ms/step - loss: 1.3891 - accuracy: 0.2416 - val_loss: 1.3858 - val_accuracy: 0.2509
Epoch 2/3
104/104 [==============================] - 1s 7ms/step - loss: 1.3870 - accuracy: 0.2573 - val_loss: 1.3836 - val_accuracy: 0.2979
Epoch 3/3
 22/104 [=====>........................] - ETA: 0s - loss: 1.3849 - accuracy: 0.2571

In [ ]:
best_model = tuner.get_best_models(1)[0]

In [ ]:
best_hyperparameters = tuner.get_best_hyperparameters(1)[0]